<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_5_tabular_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
**Module 7: Generative Adversarial Networks**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to GANs for Image and Data Generation [[Video]](https://www.youtube.com/watch?v=hZw-AjbdN5k&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_1_gan_intro.ipynb)
* Part 7.2: Train StyleGAN3 with your Own Images [[Video]](https://www.youtube.com/watch?v=R546LYsQk5M&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_2_train_gan.ipynb)
* Part 7.3: Exploring the StyleGAN Latent Vector [[Video]](https://www.youtube.com/watch?v=goQzp8QSb2s&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_3_latent_vector.ipynb)
* Part 7.4: GANs to Enhance Old Photographs Deoldify [[Video]](https://www.youtube.com/watch?v=0OTd5GlHRx4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_4_deoldify.ipynb)
* **Part 7.5: GANs for Tabular Synthetic Data Generation** [[Video]](https://www.youtube.com/watch?v=yujdA46HKwA&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_5_tabular_synthetic.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.
  Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Part 7.5: GANs for Tabular Synthetic Data Generation

Typically GANs are used to generate images. However, we can also generate tabular data from a GAN. In this part, we will use the Python tabgan utility to create fake data from tabular data. Specifically, we will use the Auto MPG dataset to train a GAN to generate fake cars.  [Cite:ashrapov2020tabular](https://arxiv.org/pdf/2010.00638.pdf)

## Installing Tabgan

Pytorch is the foundation of the tabgan neural network utility. The following code installs the needed software to run tabgan in Google Colab.

In [ ]:
# HIDE OUTPUT
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2024-04-09 06:57:56--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     214  --.-KB/s    in 0s      

2024-04-09 06:57:56 (21.9 MB/s) - ‘requirements.txt.1’ saved [214/214]

  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

Note, after installing; you may see this message:

* You must restart the runtime in order to use newly installed versions.

If so, click the "restart runtime" button just under the message. Then rerun this notebook, and you should not receive further issues.

## Loading the Auto MPG Data and Training a Neural Network

We will begin by generating fake data for the Auto MPG dataset we have previously seen. The tabgan library can generate categorical (textual) and continuous (numeric) data. However, it cannot generate unstructured data, such as the name of the automobile. Car names, such as "AMC Rebel SST" cannot be replicated by the GAN, because every row has a different car name; it is a textual but non-categorical value.

The following code is similar to what we have seen before. We load the AutoMPG dataset. The tabgan library requires Pandas dataframe to train. Because of this, we keep both the Pandas and Numpy values.

In [ ]:
!pip install --upgrade numpy

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 8.0.0 requires python-dateutil<3,>=2.8.2, but you have python-dateutil 2.8.1 which is incompatible.


In [ ]:
df_x_train

NameError: name 'df_x_train' is not defined

In [ ]:
# HIDE OUTPUT
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv("/content/small_scale_50percent_combined_genecounts.csv")

COLS_USED = ['GeneIDs','CLL Gene Counts (SRR1812702)','CLL Gene Counts (SRR1812703)',
             'CLL Gene Counts (SRR1812704)','CLL Gene Counts (SRR1812705)',
             'CLL Gene Counts (SRR1812706)','CLL Gene Counts (SRR13310942)',
             'CLL Gene Counts (SRR13310943)','CLL Gene Counts (SRR13310944)',
             'CLL Gene Counts (SRR13310945)','CLL Gene Counts (SRR13310946)','CLL Gene Counts (SRR13310947)',
             'Non-CLL Gene Counts (SRR1812751)','Non-CLL Gene Counts (SRR1812752)','Non-CLL Gene Counts (SRR1812753)']

df = df[COLS_USED]


# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df_train.drop(['GeneIDs','CLL Gene Counts (SRR1812702)'],axis=1),
    df_train['CLL Gene Counts (SRR1812702)'],
    test_size=0.10,
    shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.to_numpy()
y_train = df_y_train.to_numpy()
x_test = df_x_test.to_numpy()
y_test = df_y_test.to_numpy()


# Build the neural network
model = Sequential([
    Dense(200, input_dim=x_train.shape[1], activation='relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(12, activation='relu'),
    Dense(1)
])
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1,
     patience=5, verbose=2, mode='auto',
       restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],
         verbose=2,epochs=100)

Epoch 1/100
1989/1989 - 7s - loss: 2032668.3750 - val_loss: 6419.2188 - 7s/epoch - 4ms/step
Epoch 2/100
1989/1989 - 6s - loss: 1703906.8750 - val_loss: 5651.2534 - 6s/epoch - 3ms/step
Epoch 3/100
1989/1989 - 6s - loss: 2136716.2500 - val_loss: 2138.2146 - 6s/epoch - 3ms/step
Epoch 4/100
1989/1989 - 6s - loss: 837519.4375 - val_loss: 1209.2742 - 6s/epoch - 3ms/step
Epoch 5/100
1989/1989 - 6s - loss: 2635929.7500 - val_loss: 817.5148 - 6s/epoch - 3ms/step
Epoch 6/100
1989/1989 - 6s - loss: 1948887.2500 - val_loss: 7256.0352 - 6s/epoch - 3ms/step
Epoch 7/100
1989/1989 - 6s - loss: 2117594.5000 - val_loss: 9623.4941 - 6s/epoch - 3ms/step
Epoch 8/100
1989/1989 - 6s - loss: 1159027.8750 - val_loss: 1541.2057 - 6s/epoch - 3ms/step
Epoch 9/100
1989/1989 - 6s - loss: 890405.4375 - val_loss: 3061.8855 - 6s/epoch - 3ms/step
Epoch 10/100
Restoring model weights from the end of the best epoch: 5.
1989/1989 - 6s - loss: 1629665.6250 - val_loss: 3398.6208 - 6s/epoch - 3ms/step
Epoch 10: early stoppin

We now evaluate the trained neural network to see the RMSE. We will use this trained neural network to compare the accuracy between the original data and the GAN-generated data. We will later see that you can use such comparisons for anomaly detection. We can use this technique can be used for security systems. If a neural network trained on original data does not perform well on new data, then the new data may be suspect or fake.

In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

221/221 [==============================] - 0s 1ms/step
Final score (RMSE): 28.59221653622254


## Training a GAN for Auto MPG

Next, we will train the GAN to generate fake data from the original MPG data. There are quite a few options that you can fine-tune for the GAN. The example presented here uses most of the default values. These are the usual hyperparameters that must be tuned for any model and require some experimentation for optimal results. To learn more about tabgab refer to its paper or this [Medium article](https://towardsdatascience.com/review-of-gans-for-tabular-data-a30a2199342), written by the creator of tabgan.

In [ ]:
!pip install numpy

In [ ]:
!pip install tabgan

In [ ]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_x, gen_y = GANGenerator(gen_x_times=15, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999,
           is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse",
               "max_depth": 2,
               "max_bin": 5500,
               "learning_rate": 0.001,
               "random_state": 42,
               "n_estimators": 300,
               "force_col_wise": True,
               # Added here if LightGBM accepts it through GANGenerator
           },
           pregeneration_frac=2,
           only_generated_data=False,
          ).generate_data_pipe(df_x_train, df_y_train, df_x_test, deep_copy=True,
                               only_adversarial=False, use_adversarial=True)


Fitting CTGAN transformers for each column:   0%|          | 0/14 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5658, number of negative: 5657
[LightGBM] [Info] Total Bins 2880
[LightGBM] [Info] Number of data points in the train set: 11315, number of used features: 13
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500044 -> initscore=0.000177
[LightGBM] [Info] Start training from score 0.000177
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5658, number of negative: 5657
[LightGBM] [Info] Total Bins 2909
[LightGBM] [Info] Number of data points in the train set: 11315, number of used features: 13
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500044 -> initscore=0.000177
[LightGBM] [Info] Start training from score 0.000177
[LightGBM] [Warning] Fou

Note: if you receive an error running the above code, you likely need to restart the runtime. You should have a "restart runtime" button in the output from the second cell. Once you restart the runtime, rerun all of the cells. This step is necessary as tabgan requires specific versions of some packages.

## Evaluating the GAN Results

If we display the results, we can see that the GAN-generated data looks similar to the original. Some values, typically whole numbers in the original data, have fractional values in the synthetic data.

In [ ]:
gen_x_new =gen_x.astype(int)


In [ ]:
# Assuming df is your DataFrame
gen_x.to_csv('synthetic_10CLL_20percent_smallscale_genecounts.csv', index=False)  # Save DataFrame to CSV

from google.colab import files
files.download('synthetic_10CLL_20percent_smallscale_genecounts.csv')  # Download the file to your local system

Finally, we present the synthetic data to the previously trained neural network to see how accurately we can predict the synthetic targets.  As we can see, you lose some RMSE accuracy by going to synthetic data.

In [ ]:
# Predict
pred = model.predict(gen_x_new.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

1799/1799 [==============================] - 3s 1ms/step
Final score (RMSE): 44.03693980068866


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

# Assume x_train is your real data and gen_x is your generated data
real_labels = np.ones(len(x_train_new))  # Labels for real data
generated_labels = np.zeros(len(gen_x_new))  # Labels for generated data

# Combine the real and generated data
combined_samples = np.vstack((x_train_new, gen_x_new))
combined_labels = np.hstack((real_labels, generated_labels))

# Shuffle the combined dataset to ensure random distribution
shuffled_indices = np.random.permutation(len(combined_samples))
combined_samples = combined_samples[shuffled_indices]
combined_labels = combined_labels[shuffled_indices]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_samples, combined_labels, test_size=0.3, random_state=42)

# Train a classifier to distinguish between real and generated data
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")



Precision: 0.366072573406847
Recall: 0.16556483414520992


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

auc_roc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"AUC-ROC: {auc_roc}")
print(f"F1 Score: {f1}")

Accuracy: 0.4401100492325514
AUC-ROC: 0.43977651634369136
F1 Score: 0.22800782653835402
